In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def makeDF (tuples, header):
    '''Assumes tuples as Python tuples both empty or non empty; header as a tuple with a convention 
       as (RollNumber, Name, Exam-[name]-[max-marks], ..., Lab-[name]-[max-marks], ..., 
       Asgn-[name]-[max-marks], ..., Oth-[name]-[max-marks],)
       
       Returns a Pandas DataFrame with all NULL values replaced by Cipher, and adds a fraud column
       for figuring out cheating factor for later functions.'''
    
    #make rows, column IDs and marks as a list for DataFrame initialization
    row_index = [int(i) for i in range(1, len(tuples) + 1)]
    col_index = list(header)
    values = list(tuples)
    
    #DataFrame initialisation
    df = pd.DataFrame(tuples, row_index, col_index)
    
    #Handling of NULLs
    for col in list(df.columns):
        df[col] = df[col].fillna(value = 0)
        
    #Make a copy of last given exams marks
    df['fraud'] = 0
    df['fraud'] = df[df.columns[-2]]

    return df

def scaleMarks (df):
    '''Assumes df as a Pandas DataFrame.
    
       Returns a Pandas DataFrame with marks scaled up according to the max-marks defined in the 
       column headings'''
    
    #iterate through all columns and scale marks using apply() attribute of DataFrames
    for exam in list(df.columns):
        if len(exam.split('-')) > 2:
            df[exam] = df[exam].apply(lambda x : (x*100)/int(exam.split('-')[2]))
    
    return df

def createAvg (marks):
    '''Assumes marks as a Pandas DataFrame.
    
       Returns a DataFrame with added columns for overall weighted average, and individual exam,
       lab, assignments and other evaluations average'''
    
    #initilaize columns as zero
    marks['overall'] = 0
    marks['avgExam'] = 0
    marks['avgLab'] = 0
    marks['avgAsgn'] = 0
    marks['avgOth'] = 0
    
    #initialize count variables as zero
    exams = 0
    lab = 0
    asgn = 0
    oth = 0

    #iterate through the column list, filter and sum based on '-' as the additional columns do not have a '-'
    for exam in list(marks.columns):
        if exam.lower().startswith('exam'):
            marks['avgExam'] += marks[exam]
            exams += 1  
            
        elif exam.lower().startswith('lab'):
            marks['avgLab'] += marks[exam]
            lab += 1
            
        elif exam.lower().startswith('asgn'):
            marks['avgAsgn'] += marks[exam]
            asgn += 1
            
        elif exam.lower().startswith('oth'):
            marks['avgOth'] += marks[exam]
            oth += 1
            
        else :
            continue
    
    #weight and scale marks and divide by total number of instances of similar type counted.
    #Weights based on the strictness  and students' interest in overall exam process
    marks['overall'] = 0.5*marks['avgExam']/exams + 0.3*marks['avgLab']/lab + 0.1*marks['avgAsgn']/asgn + 0.1*marks['avgOth']/oth
    
    return marks

def createChMarks (marks):
    '''Assumes marks as a Pandas DataFrame.
       
       Returns a DataFrame with added column ChMarks which would be used further for overall cheating status'''
    
    #Not included marks for Assignments as they are done by students AT HOME
    marks['ChMarks'] = (marks['avgExam'] + marks['avgLab'] + marks['avgOth'])/3
    return marks

def variance(df):
    '''Assumes df as a Pandas DataFrame.
    
       Returns the same DataFrame with added column for variance which has variance for all scores for a particular
       student'''
    
    #Figure out first the columns to be considered for variance calculation. Used '-' as an identifier again
    ls = list(df.columns)
    buffer = []
    for i in range(len(ls)):
        if len(ls[i].split('-')) > 2:
            buffer.append(ls[i])
        else:
            continue
    
    #initialise column var with iteration based indices so as to use the power of apply() attribute
    df['var'] = [int(i) for i in range(len(df[df.columns[0]]))]
    
    #make a dummy row index for slicing DataFrame for calculation
    row_index = [int(i) for i in range(1, 1 + len(df[df.columns[0]]))]
    
    #use the value in var as an indirect reference for the whole row and use the describe() attribute to get std
    df['var'] = df['var'].apply(lambda x : (df.loc[row_index,buffer].iloc[x].describe()['std'])**2)
    
    return df

def CI(marks, column):
    '''Assumes marks as a Pandas DataFrame and column and a string.
    
       Returns the 95% confidence interval for the given data as a tuple with entries as (low, high)'''
    
    column = str(column)
    
    #CI = mean +- 2*std_error; std_error = std_deviation/sqrt(total observations)
    std_error = marks[column].describe()['std']/(len(marks['avgExam']))**0.5
    mean = marks[column].describe()['mean']
    
    return (mean - 2*std_error, mean + 2*std_error,)

def width(tup):
    '''Assumes tup as tuple.
    
       Returns an integer as the difference of 2nd and 1st values of tuple'''
    
    return tup[1] - tup[0]

def CourseStats(marks):
    '''Assumes marks as a Pandas DataFrame.
    
       Returns a tuple with values as : (course_difficulty, cheat_risk, list(cheat_flagged), 
                                         avg_marks, quartile1, quartile2, quartile3,)
                                         
       course_difficulty (str) : HIGH/MODERATE/EASY based on the weighted average and cut-off marks
       cheat_risk (str) : HIGH/MODERATE/LOW based on the spread of Assignment and Other Exam marks
       cheat_flagged (list) : A list of 5 RollNumbers who we believe with some confidence are 
                              indulged in academic malpractices in the class as a whole.
       avg_marks (str) : A range of marks where the most of students lie in between.
       quartile1, quartile2, quartile3 (int) : The stastical quartile scores for the overall analysis.'''
    
    #Calculate course difficulty based on 3rd Quartile scores of students.
    marker = marks['overall'].describe()['75%']
    if marker > 0 and marker <40 :
        course_difficulty = "HIGH"
    elif marker > 40 and marker < 75 :
        course_difficulty = "MODERATE"
    else :
        course_difficulty = "EASY"
        
    #Calculate the probability of cheating based on the width of assignment scores and other marks combined    
    cheatProb = 1 - width(CI(marks, 'avgAsgn'))/width(CI(marks, 'ChMarks'))
    if cheatProb > 0.7 and cheatProb < 1 :
        cheat_risk = "HIGH"
    elif cheatProb >0.4 and cheatProb < 0.7 :
        cheat_risk = "MODERATE"
    else :
        cheat_risk = "LOW"
    
    #Flag out top 5 students whose overall scores and assignment socres tell two different stories
    marks['cheatflagged'] = 0
    marks['cheatflagged'] = marks['avgAsgn'] - df['ChMarks']
    cheat_flagged = marks.sort_values('cheatflagged', ascending = False)['RollNumber'].iloc[1:6]
    
    #Calculate the range of marks for most students
    avg_marks = str(CI(df,'overall')[0]) + '-' + str(CI(df,'overall')[1])
    
    #Calculate quartile scores for weighted marks
    quartile1 = marks['overall'].describe()['25%']
    quartile2 = marks['overall'].describe()['50%']
    quartile3 = marks['overall'].describe()['75%']
    
    return (course_difficulty, cheat_risk, list(cheat_flagged), avg_marks, quartile1, quartile2, quartile3,)

def ExamStats(marks):
    '''Assumes marks as a Pandas DataFrame.
    
       Returns a tuple with values as : (exam_difficulty, cheat_risk, list(cheat_flagged), 
                                         avg_marks, quartile1, quartile2, quartile3,)
                                         
       exam_difficulty (str) : HIGH/MODERATE/EASY based on the exam performance
       cheat_risk (str) : HIGH/MODERATE/LOW based on the unevenness in marks
       cheat_flagged (list) : A list of 5 RollNumbers who we believe with some confidence should
                              be re-evaluated
       avg_marks (str) : A range of marks where the most of students lie in between.
       quartile1, quartile2, quartile3 (int) : The stastical quartile scores for the overall analysis.'''
    
    #Figure out the name of last exam and store it in location
    temp = list(marks.columns)
    count = 1
    for i in range(len(temp)):
        if len(temp[i].split('-')) > 2 :
            count += 1
    location = temp[count]
    
    #Calculate the difficulty based on 2nd quartile cut-offs
    marker = marks[location].describe()['50%']
    if marker > 0 and marker <40 :
        exam_difficulty = "HIGH"
    elif marker > 40 and marker < 75 :
        exam_difficulty = "MODERATE"
    else :
        exam_difficulty = "EASY"
        
    #Build the frequency table for digit occurences, add the numbers not present in DataFrame with zero occurence
    freq_df = df['fraud'].apply(lambda x : int(x%10)).value_counts()
    for i in range (10):
        try:
            if freq_df.loc[i] >= 0:
                continue
        except:
            freq_df.loc[i] = 0
    
    #Calculate the variance of the same Dataframe and figure out cheating risk
    cheat_var = freq_df.describe()['std']**2
    if cheat_var < 15 :
        cheat_risk = 'LOW'
    if cheat_var > 15 and cheat_var < 80 :
        cheat_risk = 'MODERATE'
    else:
        cheat_risk = 'HIGH'
    
    #Find the number with most occurences, sample 5 random roll numbers with that number for re-evaluation
    max_repeat = freq_df.index[0]
    marks['fraud'] = marks['fraud'].apply(lambda x : int(x%10))
    suspicious = marks[marks['fraud'] == max_repeat]['fraud']
    check_sheets_index = random.sample(range(len(suspicious)), 5)
    cheat_flagged = []
    for index in check_sheets_index:
        cheat_flagged.append(marks['RollNumber'].iloc[index])
       
    #Calculate the range of marks for most students
    avg_marks = str(CI(df,location)[0]) + '-' + str(CI(df,location)[1])
    
    #Calculate quartile scores for exam marks
    quartile1 = marks[location].describe()['25%']
    quartile2 = marks[location].describe()['50%']
    quartile3 = marks[location].describe()['75%']
    
    return (exam_difficulty, cheat_risk, cheat_flagged, avg_marks, quartile1, quartile2, quartile3,)
        

def PersistentLabels(df):
    '''Assumes df as a Pandas DataFrame.
    
       Returns a tuple with values as (consistent, moderately_varying, highly_varying,)
       
       consistent (list) : RollNumbers have almost no variation in their marks obtained so far.
       moderately_varying (list) : RollNumbers have some variation in their marks obtained so far.
       highly_varying (list) : RollNumbers have a high variation in their marks obtained so far.'''
    
    #calculate and filter the roll number list
    consistent =  list(df[df['var'] < 30]['RollNumber'])
    moderately_varying = list(df[(df['var'] > 30) & (df['var'] < 150)]['RollNumber'])
    highly_varying = list(df[df['var'] > 150]['RollNumber'])
    
    return (consistent, moderately_varying, highly_varying,)


def PerformanceLabels(df):
    '''Assumes df as a Pandas DataFrame.
    
       Returns a tuple with values as (exceptional, promising, average, needy,)
       
       exceptional (list) : RollNumbers with really good performance overall.
       promising (list) : RollNumbers who can be pushed to top with a little efforts.
       average (list) : RollNumbers who are just a few steps from failing marks and need some attention.
       needy (list) : RollNumbers who are in an immediate need of attention.'''
    
    #Calculate and filter the roll number list
    exceptional = list(df[df['overall'] > 85]['RollNumber'])
    promising = list(df[(df['overall'] < 85) & (df['overall'] > 50)]['RollNumber'])
    average = list(df[(df['overall'] < 50) & (df['overall'] > 30)]['RollNumber'])
    needy = list(df[df['overall'] < 30]['RollNumber'])
    
    return (exceptional, promising, average, needy,)

def mainFunc(df):
    df['temp'] = 1/df['overall'] + df['var']
    
    return list(df.sort_values('temp', ascending = False)['RollNumber'][0:5])


In [3]:
tuples = np.random.randn(267, 14)
headers =  ['RollNumber', 'Name', 'exam-mid-35', 'exam-end-50', 'lab-basic01-20','lab-basic02-20','lab-basic03-20','asgn-basic01-15','asgn-basic02-15','asgn-basic03-15','asgn-basic04-15','oth-quiz01-30', 'oth-quiz02-30', 'oth-quiz03-30']
roll = [i for i in range(1, 268)]
max_marks = [1, 1, 35, 50, 20, 20, 20, 15, 15, 15, 15, 30, 30, 30]
df = pd.DataFrame(tuples)
for i in range(14):
    df[i] = df[i].apply(lambda x : int((x*100)%max_marks[i]))
df.columns = headers
df['RollNumber'] = roll
df['fraud'] = 0
df['fraud'] = df['oth-quiz03-30']
print(tuples)

[[ 0.74432814 -0.37155281 -0.87426182 ... -0.71708768  0.00928787
   0.20519555]
 [ 0.11426068 -1.75810997  1.34359148 ...  0.44277902  0.3227613
  -0.71687144]
 [-1.61594007 -1.70943614  1.54681818 ... -0.37185371  1.04255503
  -1.96985387]
 ...
 [-1.29238635  0.11715301  0.37628773 ... -1.00376314  2.07713245
  -1.04292413]
 [ 0.63186084  0.71199064  1.62376024 ... -0.5306602   0.64121529
   1.90545286]
 [-0.97208869  1.52241883 -1.59584308 ... -0.78372225 -0.85207921
  -0.20284834]]


In [4]:
df.head()

,RollNumber,Name,exam-mid-35,exam-end-50,lab-basic01-20,lab-basic02-20,lab-basic03-20,asgn-basic01-15,asgn-basic02-15,asgn-basic03-15,asgn-basic04-15,oth-quiz01-30,oth-quiz02-30,oth-quiz03-30,fraud
0,1,0,17,3,8,13,16,9,6,8,0,18,0,20,20
1,2,0,29,41,7,3,14,5,14,2,8,14,2,18,18
2,3,0,14,2,6,2,4,3,7,0,0,22,14,13,13
3,4,0,26,15,17,4,9,0,12,2,3,25,26,23,23
4,5,0,16,35,7,19,18,8,5,11,2,22,6,7,7


In [5]:
df = scaleMarks(df)
df.head()

,RollNumber,Name,exam-mid-35,exam-end-50,lab-basic01-20,lab-basic02-20,lab-basic03-20,asgn-basic01-15,asgn-basic02-15,asgn-basic03-15,asgn-basic04-15,oth-quiz01-30,oth-quiz02-30,oth-quiz03-30,fraud
0,1,0,48.571429,6.0,40.0,65.0,80.0,60.000000,40.000000,53.333333,0.000000,60.000000,0.000000,66.666667,20
1,2,0,82.857143,82.0,35.0,15.0,70.0,33.333333,93.333333,13.333333,53.333333,46.666667,6.666667,60.000000,18
2,3,0,40.000000,4.0,30.0,10.0,20.0,20.000000,46.666667,0.000000,0.000000,73.333333,46.666667,43.333333,13
3,4,0,74.285714,30.0,85.0,20.0,45.0,0.000000,80.000000,13.333333,20.000000,83.333333,86.666667,76.666667,23
4,5,0,45.714286,70.0,35.0,95.0,90.0,53.333333,33.333333,73.333333,13.333333,73.333333,20.000000,23.333333,7


In [6]:
df = createAvg(df)
df = createChMarks(df)
df = variance(df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:117: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [7]:
df.head()

,RollNumber,Name,exam-mid-35,exam-end-50,lab-basic01-20,lab-basic02-20,lab-basic03-20,asgn-basic01-15,asgn-basic02-15,asgn-basic03-15,...,oth-quiz02-30,oth-quiz03-30,fraud,overall,avgExam,avgLab,avgAsgn,avgOth,ChMarks,var
0,1,0,48.571429,6.0,40.0,65.0,80.0,60.000000,40.000000,53.333333,...,0.000000,66.666667,20,40.198413,54.571429,185.0,153.333333,126.666667,122.079365,858.873153
1,2,0,82.857143,82.0,35.0,15.0,70.0,33.333333,93.333333,13.333333,...,6.666667,60.000000,18,61.825397,164.857143,120.0,193.333333,113.333333,132.730159,520.979798
2,3,0,40.000000,4.0,30.0,10.0,20.0,20.000000,46.666667,0.000000,...,46.666667,43.333333,13,24.111111,44.000000,60.0,66.666667,163.333333,89.111111,1084.900021
3,4,0,74.285714,30.0,85.0,20.0,45.0,0.000000,80.000000,13.333333,...,86.666667,76.666667,23,52.126984,104.285714,150.0,113.333333,246.666667,166.984127,776.963513
4,5,0,45.714286,70.0,35.0,95.0,90.0,53.333333,33.333333,73.333333,...,20.000000,23.333333,7,59.150794,115.714286,220.0,173.333333,116.666667,150.793651,1013.586116


In [8]:
CourseStats(df)

('MODERATE',
 'LOW',
 [257, 236, 135, 9, 235],
 '45.568187333951826-48.40131507375284',
 40.226190476190474,
 47.06349206349206,
 53.980158730158735)

In [9]:
df.head()

,RollNumber,Name,exam-mid-35,exam-end-50,lab-basic01-20,lab-basic02-20,lab-basic03-20,asgn-basic01-15,asgn-basic02-15,asgn-basic03-15,...,oth-quiz03-30,fraud,overall,avgExam,avgLab,avgAsgn,avgOth,ChMarks,var,cheatflagged
0,1,0,48.571429,6.0,40.0,65.0,80.0,60.000000,40.000000,53.333333,...,66.666667,20,40.198413,54.571429,185.0,153.333333,126.666667,122.079365,858.873153,31.253968
1,2,0,82.857143,82.0,35.0,15.0,70.0,33.333333,93.333333,13.333333,...,60.000000,18,61.825397,164.857143,120.0,193.333333,113.333333,132.730159,520.979798,60.603175
2,3,0,40.000000,4.0,30.0,10.0,20.0,20.000000,46.666667,0.000000,...,43.333333,13,24.111111,44.000000,60.0,66.666667,163.333333,89.111111,1084.900021,-22.444444
3,4,0,74.285714,30.0,85.0,20.0,45.0,0.000000,80.000000,13.333333,...,76.666667,23,52.126984,104.285714,150.0,113.333333,246.666667,166.984127,776.963513,-53.650794
4,5,0,45.714286,70.0,35.0,95.0,90.0,53.333333,33.333333,73.333333,...,23.333333,7,59.150794,115.714286,220.0,173.333333,116.666667,150.793651,1013.586116,22.539683


In [10]:
ExamStats(df)

('MODERATE',
 'MODERATE',
 [24, 5, 25, 32, 33],
 '43.641565564728545-50.915238430277704',
 20.0,
 46.666666666666664,
 75.0)

In [19]:
PersistentLabels(df)

([],
 [],
 [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,

In [20]:
PerformanceLabels(df)

([],
 [2,
  3,
  6,
  10,
  12,
  14,
  20,
  21,
  22,
  24,
  25,
  28,
  33,
  35,
  41,
  44,
  46,
  47,
  48,
  55,
  56,
  60,
  61,
  62,
  63,
  71,
  76,
  81,
  84,
  85,
  87,
  88,
  89,
  94,
  95,
  98,
  103,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  115,
  116,
  118,
  119,
  121,
  122,
  125,
  126,
  128,
  129,
  130,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  144,
  146,
  148,
  150,
  154,
  158,
  159,
  160,
  162,
  167,
  168,
  169,
  170,
  173,
  174,
  175,
  177,
  178,
  179,
  180,
  182,
  188,
  189,
  191,
  192,
  194,
  196,
  197,
  198,
  203,
  205,
  210,
  212,
  213,
  219,
  223,
  225,
  226,
  228,
  229,
  233,
  235,
  236,
  239,
  240,
  241,
  242,
  248,
  249,
  251,
  254,
  257,
  263,
  264,
  265,
  267],
 [1,
  5,
  7,
  8,
  9,
  11,
  13,
  15,
  16,
  18,
  19,
  23,
  26,
  27,
  29,
  30,
  32,
  34,
  36,
  37,
  38,
  39,
  40,
  42,
  43,
  45,
  49,
  51,
  52,
  53,
  54,
  58,
  59,
  64,
  65,
  

In [21]:
mainFunc(df)

[205, 102, 49, 220, 262]